## Preprocessing

In [42]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

In [33]:
DATA_ROOT = Path("../data") / "jigsaw"

train = pd.read_csv(DATA_ROOT / "train.csv").fillna("sterby")
test = pd.read_csv(DATA_ROOT / "test_proced.csv").fillna("sterby")

In [34]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [44]:
# Prepare .txt for training the fastText model
# 
# __label_1 text_content \n
# __label_1 label_2 label_3 text_content 

def to_txt_labels(df, filename):
    with open(DATA_ROOT / (filename + ".txt"), "w") as txtfile:
        for index, row in df.iterrows():
            string = ''
            # loop through ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
            for col in df.columns.tolist()[2:]:
                string += (('__label__' + col + ' ') if (row[col] == 1) else '')
            if string == '':
                string += 'label_nontoxic' + ' '
            txtfile.write(string + row['comment_text'] + '\n')
            
to_txt_labels(train, 'train')
to_txt_labels(test, 'test')

# Training

fastText installed from
```
wget https://github.com/facebookresearch/fastText/archive/v0.1.0.zip
unzip v0.1.0.zip
cd fastText-0.1.0
make
```
Training (in terminal)
```
./fasttext supervised -input train.txt -output ft_model -lr 1.0 -epoch 25
./fasttext test ft_model.bin test.txt
```
To query the embedding of a word:
```
echo "enviroment" | ./fasttext print-word-vectors ft_model.bin
```